In [ ]:
import pycuda.autoinit
import numpy as np
from pycuda import gpuarray
import libcudnn, ctypes
import pycuda.driver as drv
import pandas as pd
from pycuda.compiler import SourceModule

In [ ]:
# Define some globals. Should be defined in a constructor for a sequential class.

# Initialize the cuDNN context
cudnn_context = libcudnn.cudnnCreate()

# Set some options and tensor dimensions
tensor_format = libcudnn.cudnnTensorFormat['CUDNN_TENSOR_NCHW']
data_type = libcudnn.cudnnDataType['CUDNN_DATA_FLOAT']
data_type_np = np.float32
softmax_mode = libcudnn.cudnnSoftmaxMode['CUDNN_SOFTMAX_MODE_INSTANCE']
softmax_algo = libcudnn.cudnnSoftmaxAlgorithm['CUDNN_SOFTMAX_ACCURATE']

In [ ]:
# Some helper functions for analysis and testing
def analyze_prediction(preds, actuals):
    
    n_data, _ = preds.shape
    preds_sparse = np.argmax(preds, axis=1)
    actuals_sparse = np.argmax(actuals, axis=1)
    return sum(preds_sparse == actuals_sparse) / n_data

In [ ]:
def simple_data4():
    
    n_features = 4
    n_classes = 2
    n_data = 10
    
    X_train = np.random.rand(n_data, n_features)
    idx = np.array([1 if X_train[i][0] > 0.5 else 0 for i in range(len(X_train))])
    Y_train = np.zeros((n_data, n_classes))
    for i, row in enumerate(Y_train):
        Y_train[i][idx[i]] = 1

    return X_train, Y_train

def simple_data_fixed():
    
    X_train = np.array([[0.16135288, 0.30464995, 0.22296312, 0.2271508 ],
       [0.44617278, 0.25550829, 0.31906067, 0.53594914],
       [0.55487757, 0.36622737, 0.60981441, 0.55972613],
       [0.99677183, 0.75638836, 0.06633571, 0.60571115],
       [0.35337454, 0.34625126, 0.7179572 , 0.80923418],
       [0.62548399, 0.69020976, 0.97985429, 0.16891352],
       [0.59629351, 0.5056652 , 0.50843348, 0.7934896 ],
       [0.46224334, 0.70235517, 0.26416074, 0.97176787],
       [0.33548574, 0.64162364, 0.37395584, 0.32417411],
       [0.21933534, 0.96979702, 0.41459635, 0.57283444]], dtype=np.float32)
    
    Y_train = np.array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=np.float32)
    return X_train, Y_train

def iris_data():
    
    df = pd.read_csv("data/iris.csv") #.sample(10)
    Y_train = pd.get_dummies(df["species"]).to_numpy()
    X_train = df.drop(columns=["species"]).to_numpy()

    return X_train, Y_train

#train_data_np, train_labels_np = simple_data_fixed()
train_data_np, train_labels_np = iris_data()
n_train, n_features = train_data_np.shape
_, n_classes = train_labels_np.shape

In [ ]:
class sequential_nn:
    def __init__(self, cudnn_context, X_train, X_desc, Y_train, 
                 Y_desc, iterations, learning_rate, n_features, n_layers, n_classes,
                 data_type_np, tensor_format, data_type):
        
        if n_layers > 1:
            print("Support for greater than 1 layer not yet supported.")
            return
        
        self.cudnn_context = cudnn_context
        self.n_features = n_features
        self.n_layers = n_layers
        self.sequential_layers = [None]*n_layers
        self.layer_sizes = [0]*(n_layers+1)
        self.layer_sizes[0] = n_features
        
        # Define loss as cross entropy
        self.loss = cross_entropy_loss(cudnn_context, n_classes, 
                                       n_train, data_type_np, tensor_format, data_type)
        
        # ToDo: allocate more layers
        
        # Set the output softmax layer
        self.layer_sizes[-1] = n_classes
        self.sequential_layers[-1] = softmax(cudnn_context, 
                                             n_classes, 
                                             n_train, 
                                             n_features, 
                                             data_type_np, 
                                             tensor_format, 
                                             data_type)
        
        # Fit the model parameters to the data
        self._fit(X_train, X_desc, Y_train, Y_desc, iterations, learning_rate)  # Pass in only n_train and do this later?
    
    def _fit(self, X_train, X_train_desc, Y_train, Y_train_desc, iterations, learning_rate):

        for i in range(iterations):
            
            _, _ = self.forward(X_train, X_train_desc)
            self.backward(X_train, X_train_desc, Y_train, Y_train_desc)
            self._apply_updates(learning_rate)
            
    def forward(self, X, X_desc):
        
        X_in = X
        X_in_desc = X_desc
        for layer in range(self.n_layers):
            current_layer = self.sequential_layers[layer]
            X_in, X_in_desc = current_layer.forward(X_in, X_in_desc)
        return X_in, X_in_desc
    
    def backward(self, X_train, X_train_desc, Y_train, Y_train_desc):
        
        last_layer = self.sequential_layers[-1]
        preds = last_layer.Y
        dY, dY_desc = self.loss.derivative(Y_train, preds)
        
        for layer in range(self.n_layers-1, -1, -1):
            current_layer = self.sequential_layers[layer]
            
            if layer > 0:
                previous_layer = self.sequential_layers[layer-1]
                X = previous_layer.Y
                X_desc = previous_layer.Y_desc
            else:
                X = X_train
                X_desc = X_train_desc
            
            dY, dY_desc = current_layer.backward(dY, dY_desc, X, X_desc)
    
    def _apply_updates(self, learning_rate):
    
        for layer in range(self.n_layers):
            
            current_layer = self.sequential_layers[layer]
            current_layer.apply_updates(learning_rate)


In [ ]:
class layer:
    def __init__(self, cudnn_context, n_train, data_type_np, tensor_format, data_type):
        self.cudnn_context = cudnn_context
        self.n_train = n_train
        self.data_type_np = data_type_np
        self.tensor_format = tensor_format
        self.data_type = data_type

    def forward(self, X, X_desc):
        pass
    
    def backward(self, dY, dY_desc, X, X_desc):
        pass


class softmax(layer):
    def __init__(self, cudnn_context, n_classes, n_train, n_input, data_type_np, tensor_format, data_type):
        
        super().__init__(cudnn_context, n_train, data_type_np, tensor_format, data_type)
        self.n_classes = n_classes
        self.n_input = n_input
        
        #W_np = np.ones((n_classes, n_input))  # !!!! Make sure to initialize better !!!!
        #b_np = np.ones((n_classes, 1))
        W_np = np.random.normal(size=(n_classes, n_input), scale=0.1)
        b_np = np.random.normal(size=(n_classes, 1), scale=0.5)
        self.W = gpuarray.to_gpu(W_np)
        self.b = gpuarray.to_gpu(b_np)
        self.W_desc = libcudnn.cudnnCreateTensorDescriptor()
        libcudnn.cudnnSetTensor4dDescriptor(self.W_desc, tensor_format, data_type, 
                                            n_classes, n_input, 1, 1)
        self.b_desc = libcudnn.cudnnCreateTensorDescriptor()
        libcudnn.cudnnSetTensor4dDescriptor(self.b_desc, tensor_format, data_type, 
                                            n_classes, 1, 1, 1)
        
        # Output
        self.Y = gpuarray.empty((n_train, n_classes, 1, 1), data_type_np)
        self.Y_desc = libcudnn.cudnnCreateTensorDescriptor()
        libcudnn.cudnnSetTensor4dDescriptor(self.Y_desc, tensor_format, data_type, 
                                            n_train, n_classes, 1, 1)
        self.dX = None  # Need to implement!
        self.dX_desc = None
        
        # Updates
        self.weight_updates = gpuarray.empty((n_classes, n_input+1, 1, 1), data_type_np)
        self.weight_updates_desc = libcudnn.cudnnCreateTensorDescriptor()
        libcudnn.cudnnSetTensor4dDescriptor(self.weight_updates_desc, tensor_format, data_type, 
                                            n_classes, n_input+1, 1, 1)
        
        # Working space
        self.z = gpuarray.empty((n_train, n_classes, 1, 1), data_type_np)
        self.z_desc = libcudnn.cudnnCreateTensorDescriptor()
        libcudnn.cudnnSetTensor4dDescriptor(self.z_desc, tensor_format, data_type, 
                                            n_train, n_classes, 1, 1)
        self.softmax_grad = gpuarray.empty((n_train, n_classes, 1, 1), data_type_np)
        self.softmax_grad_desc = libcudnn.cudnnCreateTensorDescriptor()
        libcudnn.cudnnSetTensor4dDescriptor(self.softmax_grad_desc, tensor_format, data_type, 
                                            n_train, n_classes, 1, 1)

    def forward(self, X, X_desc):
        
        # Note: X = layer_output, Y = output_pred

        # ToDo: Get rid of numpy usage
        # Apply the linear combination of weights and inputs
        W_np = self.W.get().reshape((self.n_classes, self.n_input))
        b_np = self.b.get().reshape((self.n_classes, 1))
        X_np = X.get()

        # The matrix here is interpreted as the transpose
        z_np = (np.matmul(W_np, X_np.T) + b_np).T.reshape((self.n_classes, n_train, 1, 1))
        z = gpuarray.to_gpu(z_np.astype(data_type_np))
        self.z = z  #Note: This is only done because of numpy

        # Apply the forward softmax
        libcudnn.cudnnSoftmaxForward(handle=self.cudnn_context,
                                     algorithm=softmax_algo, 
                                     mode=softmax_mode, 
                                     alpha=1.0,
                                     srcDesc=self.z_desc,
                                     srcData=self.z.ptr,
                                     beta=0.0,
                                     destDesc=self.Y_desc,
                                     destData=self.Y.ptr)
        return self.Y, self.Y_desc
    
    def backward(self, dY, dY_desc, X, X_desc):
        
        # Note: Y = output_pred, dY = d_entropy, X = prev_layer_output, dX = ????
        
        # Calculate the backward softmax
        libcudnn.cudnnSoftmaxBackward(handle=self.cudnn_context,
                                      algorithm=softmax_algo,
                                      mode=softmax_mode,
                                      alpha=1.0,
                                      srcDesc=self.Y_desc,
                                      srcData=self.Y.ptr,
                                      srcDiffDesc=dY_desc,
                                      srcDiffData=dY.ptr,
                                      beta=0.0,
                                      destDiffDesc=self.softmax_grad_desc,
                                      destDiffData=self.softmax_grad.ptr)

        # Calculate the gradient updates
        # ToDo: remove numpy usage
        softmax_grad_np = self.softmax_grad.get().reshape((self.n_train, self.n_classes))
        weight_updates_np = np.zeros((self.n_classes, self.n_input+1))
        X_aug_np = np.concatenate([X.get(), np.ones((self.n_train, 1))], axis=1)
        for i in range(self.n_classes):
            weight_updates_np[i, :] = np.mean(X_aug_np * softmax_grad_np[:, i][:, np.newaxis], axis=0)

        weight_updates = gpuarray.to_gpu(weight_updates_np.reshape(self.n_classes, self.n_input+1, 1, 1).astype(self.data_type_np))
        self.weight_updates = weight_updates  # Note: only needed for numpy
        
        return self.dX, self.dX_desc

    def apply_updates(self, learning_rate):

        # ToDo: remove numpy usage
        weight_updates_np = self.weight_updates.get().reshape(self.n_classes, self.n_input+1)
        W_np = self.W.get().reshape(self.n_classes, self.n_input)
        b_np = self.b.get().reshape(self.n_classes, 1)

        W_updates_np = weight_updates_np[:, 0:self.n_input]
        b_updates_np = weight_updates_np[:, -1].reshape(self.n_classes, 1)

        W_np_new_np = W_np - learning_rate*W_updates_np
        b_np_new_np = b_np - learning_rate*b_updates_np

        self.W = gpuarray.to_gpu(W_np_new_np.reshape(self.n_classes, self.n_input, 1, 1).astype(self.data_type_np))
        self.b = gpuarray.to_gpu(b_np_new_np.reshape(self.n_classes, 1, 1, 1).astype(self.data_type_np))


class cross_entropy_loss:
    def __init__(self, cudnn_context, n_classes, n_train, data_type_np, tensor_format, data_type):
        self.cudnn_context = cudnn_context
        self.data_type_np = data_type_np
        self.tensor_format = tensor_format
        self.data_type = data_type
        self.n_classes = n_classes
        self.n_train = n_train
        
        # Define CUDA related data
        module = SourceModule(open("kernels.cu", "r").read())
        self.kernel_deriv_entropy = module.get_function("deriv_entropy")
        self.n_classes_c = np.int32(n_classes)
        self.n_train_c = np.int32(n_train)
        self.block_size = 256
        
        self.dX = gpuarray.empty((n_train, n_classes, 1, 1), data_type_np)
        self.dX_desc = libcudnn.cudnnCreateTensorDescriptor()
        libcudnn.cudnnSetTensor4dDescriptor(self.dX_desc, tensor_format, data_type, 
                                            n_train, n_classes, 1, 1)
        
    def calc_loss(self, X, X_desc):
        # Note: should have X = output from previous layer, Y = output of this layer
        
        #Y_np = X.get()  # Can we copy from device to device?
        #self.Y = gpuarray.to_gpu(Y_np)  # Just because of numpy
        return None
    
    def derivative(self, Y_train, preds):
        # Note: should have Y = Y_train, preds = output_pred, dX = d_entropy
        
        self.kernel_deriv_entropy(self.n_train_c,
                                  self.n_classes_c,
                                  Y_train,
                                  preds,
                                  self.dX,
                                  block=(self.n_classes, 1, 1),
                                  grid=(self.block_size, 1, 1))
        
        return self.dX, self.dX_desc


In [ ]:
# Format input data
X_train = gpuarray.to_gpu(train_data_np.astype(data_type_np))
X_desc = libcudnn.cudnnCreateTensorDescriptor()
libcudnn.cudnnSetTensor4dDescriptor(X_desc, tensor_format, data_type, 
                                    n_train, n_features, 1, 1)
Y_train = gpuarray.to_gpu(train_labels_np.astype(data_type_np))
Y_desc = libcudnn.cudnnCreateTensorDescriptor()
libcudnn.cudnnSetTensor4dDescriptor(Y_desc, tensor_format, data_type, 
                                    n_train, n_classes, 1, 1)


model = sequential_nn(cudnn_context, X_train, X_desc, Y_train, 
                      Y_desc, 500, 0.01, n_features, 1, n_classes,
                      data_type_np, tensor_format, data_type)
preds, preds_desc = model.forward(X_train, X_desc)

analyze_prediction(preds.get().reshape((n_train, n_classes)), 
                   Y_train.get().reshape((n_train, n_classes)))


In [ ]:
preds.get().reshape((n_train, n_classes))

In [ ]:
Y_train.get().reshape((n_train, n_classes))

In [ ]:
model.sequential_layers[0].weight_updates.get().T

In [ ]:
model.sequential_layers[0].Y.get().T

In [ ]:
model.sequential_layers[0].W.get().T

In [ ]:
model.sequential_layers[0].b.get().T